In [60]:
import nltk
import pandas as pd

In [61]:
training_data=pd.read_csv('training_twitter_x_y_train.csv')
x_test=pd.read_csv('test_twitter_x_test.csv')

In [62]:
training_data.shape

(10980, 12)

In [63]:
#training_data

In [64]:
training_data.isnull().sum()

tweet_id                      0
airline_sentiment             0
airline                       0
airline_sentiment_gold    10949
name                          0
negativereason_gold       10956
retweet_count                 0
text                          0
tweet_coord               10204
tweet_created                 0
tweet_location             3550
user_timezone              3577
dtype: int64

In [65]:
training_data.airline_sentiment.value_counts()

negative    6851
neutral     2327
positive    1802
Name: airline_sentiment, dtype: int64

In [66]:
sentiments=training_data.airline_sentiment
text=training_data.text
x_test=training_data.text

documents=[]

for i in range(len(sentiments)):
    documents.append((text[i],sentiments[i]))
documents[4]
x_test[3]

"@SouthwestAir @dultch97 that's horse radish 😤🐴"

In [67]:
from nltk.corpus import stopwords
from string import punctuation
stop_words=list(punctuation)+stopwords.words('english')

In [68]:
len(stop_words)

211

In [69]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [70]:
def simplify_pos_tag(tag):
    if tag.startswith('N'):
        return 'n'
    if tag.startswith('J'):
        return 'a'
    if tag.startswith('R'):
        return 'r'
    if tag.startswith('V'):
        return 'v'
    return 'n'

In [71]:
import re # Regular expression

def hasNumbers(text):
    return bool(re.search('[0-9]+',text))

def clean_text(text):
    result=[]
    for word in text.split():
        word=word.strip(punctuation)
        if ((word.lower() not in stop_words) and (not hasNumbers(word.lower())) and (len(word.lower())>3)):
            posTag=pos_tag([word.lower()])[0][1]
            lemmatized_word=lemmatizer.lemmatize(word.lower(),simplify_pos_tag(posTag))
            result.append(lemmatized_word)
    return result

In [72]:
documents_modified=[(clean_text(text),sentiment) for text,sentiment in documents]
x_test=[clean_text(text) for text in x_test]

In [73]:
pos_tag(["words"])[0][1]
lemmatizer.lemmatize('words','a')

'words'

In [74]:
print(documents_modified[8])
print(documents[8])

(['united', 'frankly', 'bad', 'customer', 'service', 'ever', 'problem', 'happen', 'deal', 'defines', 'company', 'never', 'united'], 'negative')
('@united frankly worse customer service ever. Problems will happen, how you deal defines a company. Never again United.', 'negative')


In [75]:
print(x_test[3])

['southwestair', "that's", 'horse', 'radish']


In [76]:
s='min'
s=s.strip(punctuation)
s

'min'

In [77]:
hasNumbers('a0:')

True

In [78]:
import numpy as np
# Coverting into X and Y
Y_train = [sentiment for text,sentiment in documents_modified]
#Y_train = np.array(Y_train).reshape(-1,1)
X_train = [" ".join(text) for text,sentiment in documents_modified]
X_test = [" ".join(text) for text in x_test]

In [79]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold,cross_val_score

In [80]:
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
#x_axis=[]
#y_axis=[]
#max_score=0
#k=0
#    for i in range(10,5000):
#        print(i)
#        cv=CountVectorizer(max_features=i) # Sparse matrix will have i features
#        X_train_features=cv.fit_transform(X_train) # x_train_features is a sparse matrix 
#        clf=MultinomialNB()
#        clf.fit(X_train_features,Y_train)  
#        score=cross_val_score(clf,X_train_features,Y_train,cv=KFold(n_splits=5,shuffle=True,random_state=0))
#        x_axis.append(i)
#        y_axis.append(score.mean())
#plt.plot(x_axis,y_axis)
#print(plt.show())
cv=CountVectorizer(max_features=2000,ngram_range=(1,2)) # By K-Cross validation, k=2000 is the most optimum value
X_train_features = cv.fit_transform(X_train)
X_test_features = cv.transform(X_test)
clf=MultinomialNB()
clf.fit(X_train_features,Y_train)
clf.predict(X_test_features)

array(['negative', 'positive', 'positive', ..., 'negative', 'negative',
       'negative'], dtype='<U8')

In [81]:
X_train_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [109]:
#import csv
#csv_file=open('twitter_prediction.csv','w') # we want to write to this file
#csv_writer=csv.writer(csv_file)
#csv_writer.writerow(['predictions']) # Column names
#for i in range(X_test_features.shape[0]):
#    csv_writer.writerow([clf.predict(X_test_features[i])])

In [112]:
Y_pred=clf.predict(X_test_features)

In [113]:
Y_pred

array(['negative', 'positive', 'positive', ..., 'negative', 'negative',
       'negative'], dtype='<U8')

In [114]:
clf.score(X_train_features,Y_train)

0.7961748633879782